#Evaluación, Simulación y Validación de Idoneidad

Este tema es interesante por que podemos hablar de cosas como: partición de la muestra, programación de las soluciones, sobre y supra parametrización (overfitting y underfitting), validación cruzada, bootstrapping, etc.

##Partición de Muestras y validación cruzada.
Estos dos temas son importantes, primer debemos entender ¿Porqué realizamos esto? (Tomen notas)


###Partición de muestras
La idea es simple, ¿Cómo dividir un dataset entre el conjunto de prueba y el conjunto de entrenamiento? Esto es super sencillo en Python.

In [ ]:
#Hagamos algunas cosas simples con Scikit-Learn
#Empecemos con sets de prueba y entrenamiento.
#Este es el código base para particionar en test y train sets.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#Tenemos que insertar un conjunto de datos, separaremos el 20% como el test set y el random_state es
#Para tener reproducibilidad.

A continuación un breve ejemplo para que vean cómo se utiliza esta función:

In [ ]:
#Veamos un ejemplo:
#Librerias:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Generamos datos:
np.random.seed(0)
X = np.random.rand(100, 2)
# Clasificación simple (1 si x1 + x2 > 1, 0 en caso contrario)
y = (X[:, 0] + X[:, 1] > 1).astype(int)

# Definimos el número de particiones
particiones = 5

# Creamos subplots para visualizar las particiones
fig, axes = plt.subplots(1, particiones, figsize=(15, 3))

# Realizamos múltiples particiones y visualizamos
for i in range(particiones):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i * 42)  # Diferente semilla para cada partición

    # Visualizamos los puntos de entrenamiento y prueba en el scatterplot
    ax = axes[i]
    ax.scatter(X_train[y_train == 1, 0], X_train[y_train == 1, 1], label='Clase 1', marker='o')
    ax.scatter(X_train[y_train == 0, 0], X_train[y_train == 0, 1], label='Clase 0', marker='x')
    ax.scatter(X_test[y_test == 1, 0], X_test[y_test == 1, 1], label='Prueba - Clase 1', marker='s')
    ax.scatter(X_test[y_test == 0, 0], X_test[y_test == 0, 1], label='Prueba - Clase 0', marker='d')

    ax.set_title(f'Partición {i+1}')
    ax.set_xlabel('Característica 1')
    ax.set_ylabel('Característica 2')
    if i == 0:
        ax.legend(loc='upper right')
    else:
        ax.legend().set_visible(False)

plt.tight_layout()
plt.show()

plt.tight_layout()
plt.show()


###Validación Cruzada
La validación cruzada es un método de análisis que sirve para evaluar un modelo, utiliza el mismo proceso de partición de la DB en entrenamiento y prueba y simplemente lo repite K veces. ¿Para qué sirve utilizar este método?

In [ ]:
## Este sería el código base para utilizar la validación cruzada
#(sigue siendo parte de la librería scikit.learn)
from sklearn.model_selection import cross_val_score
#Esta función sirve para realizar la validación cruzada.
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print("Scores:", scores)
print("Promedio:", scores.mean())

Ahora veamos un ejemplo visual al respecto de la validación cruzada.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

# Generamos datos de ejemplo
X, y = make_classification(n_samples=100, n_features=2,n_informative=2,n_redundant=0, random_state=345)

# Definimos un clasificador (puedes usar cualquier modelo)
model = RandomForestClassifier(n_estimators=10, random_state=2342)

# Realizamos validación cruzada con 5 pliegues
k = 5
scores = cross_val_score(model, X, y, cv=k, scoring='accuracy')
####Podemos analizar con CV cosas como
# 'f1' para el puntaje F1
# 'recall' para la sensibilidad
# 'precision' para la precisión.
# 'roc_auc' para calcular el área bajo la curva de la curva ROC.

# Gráfico de barras para visualizar los resultados
plt.bar(range(1, k + 1), scores)
plt.xlabel('Pliegues')
plt.ylabel('Exactitud (Accuracy)')
plt.title('Resultados de Validación Cruzada (Accuracy)')
plt.show()


## Overfitting (Sobreajuste) y Underfitting (Subajuste)
Los pecados mortales de la modelación.

Underfitting: El modelo es demasiado sencillo para representar adecuadamente los datos y no puede hacer predicciones precisas.

Overfitting: El modelo es demasiado complejo y se adapta demasiado a los datos de entrenamiento, perdiendo su capacidad de generalización.

In [ ]:
#Underfitting: Subajuste
#Utilizando de forma incorrecta un modelo lineal simple.
#Librerías
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Generamos datos de ejemplo
np.random.seed(0)
X = np.sort(5 * np.random.rand(80, 1), axis=0)
y = np.sin(X).ravel() + np.random.rand(80)

#Partición de sets de datos:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=311)

# Ajustamos un modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

#Revisión y cálculo del MSE (Error cuadrado medio)
y_pred = model.predict(X_test)
mse_underfit = mean_squared_error(y_test, y_pred)
print(f"MSE - Underfit: {mse_underfit}")

# Visualizamos los datos y la predicción del modelo
plt.scatter(X, y, color='darkorange', label='Datos')
plt.plot(X, model.predict(X), color='cornflowerblue', label='Modelo')
plt.xlabel('Característica')
plt.ylabel('Etiqueta')
plt.title('Underfitting: Regresión Lineal Simple')
plt.legend(loc='best')
plt.show()


In [ ]:
#Overfitting: Un modelo hiperparametrizado
#árbol de decisión con alta complejidad.
#Librerías
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor

# Generamos datos de ejemplo
np.random.seed(0)
X = np.sort(5 * np.random.rand(80, 1), axis=0)
y = np.sin(X).ravel() + np.random.rand(80)

#Partición de sets de datos:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=311)

# Ajustamos un modelo de árbol de decisión altamente complejo
model = DecisionTreeRegressor(max_depth=20)
model.fit(X_train, y_train)

#Cálculo del MSE.
y_pred = model.predict(X_test)
mse_overfit = mean_squared_error(y_test, y_pred)
print(f"MSE - Overfit: {mse_overfit}")

# Visualizamos los datos y la predicción del modelo
X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]
y_pred = model.predict(X_test)
plt.scatter(X, y, color='darkorange', label='Datos')
plt.plot(X_test, y_pred, color='cornflowerblue', label='Modelo')
plt.xlabel('Característica')
plt.ylabel('Etiqueta')
plt.title('Overfitting: Árbol de Decisión Complejo')
plt.legend(loc='best')
plt.show()

##Métricas de evaluación
Hasta ahora todo ha sido muy bueno con el manejo de ciertos números que nos sirven para definir si un modelo hizo o no la "chamba". Recordaremos puntajes como exactitud, precisión, sensibilidad y puntaje F1.

In [ ]:
#Los diversos puntajes que podemos obtener.
#Aquí no hay mucho que decir, ya los han calculado (clase pasada).
#Recuerden que este código les sirve para ver qué es lo que hay y cómo se implementa:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(cm)


In [ ]:
#Este es el ejemplo:
#Librerías
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Generamos datos de ejemplo
X, y = make_classification(n_samples=1000, n_features=20, random_state=433)
print(X.shape)
print(y.shape)
# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=82)

# Entrenamos dos modelos: Regresión Logística y Random Forest
model_logistic = LogisticRegression()
model_rf = RandomForestClassifier()

model_logistic.fit(X_train, y_train)
model_rf.fit(X_train, y_train)

# Realizamos predicciones en el conjunto de prueba
y_pred_logistic = model_logistic.predict(X_test)
y_pred_rf = model_rf.predict(X_test)

# Calculamos las métricas para el modelo de Regresión Logística
acc_logistic = accuracy_score(y_test, y_pred_logistic)
precision_logistic = precision_score(y_test, y_pred_logistic)
recall_logistic = recall_score(y_test, y_pred_logistic)
f1_logistic = f1_score(y_test, y_pred_logistic)

print("Métricas para Regresión Logística:")
print("Accuracy:", acc_logistic)
print("Precision:", precision_logistic)
print("Recall:", recall_logistic)
print("F1 Score:", f1_logistic)


# Calculamos las métricas para el modelo Random Forest
acc_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

# Calculamos y mostramos las métricas para el modelo Random Forest
print("\nMétricas para Random Forest:")
print("Accuracy:", acc_rf)
print("Precision:", precision_rf)
print("Recall:", recall_rf)
print("F1 Score:", f1_rf)


# Creamos y mostramos las matrices de confusión con Seaborn
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
cm_logistic = confusion_matrix(y_test, y_pred_logistic)
sns.heatmap(cm_logistic, annot=True, fmt="d", cmap="Blues")
plt.title("Matriz de Confusión - Regresión Logística")

plt.subplot(1, 2, 2)
cm_rf = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm_rf, annot=True, fmt="d", cmap="Greens")
plt.title("Matriz de Confusión - Random Forest")

plt.show()

# Creamos gráficos de barras para comparar las métricas
models = ["Regresión Logística", "Random Forest"]
acc_scores = [acc_logistic, acc_rf]
precision_scores = [precision_logistic, precision_rf]
recall_scores = [recall_logistic, recall_rf]
f1_scores = [f1_logistic, f1_rf]

plt.figure(figsize=(10, 6))
x = np.arange(len(models))
bar_width = 0.2

plt.bar(x, acc_scores, width=bar_width, label="Accuracy", align="center", alpha=0.7)
plt.bar(x + bar_width, precision_scores, width=bar_width, label="Precision", align="center", alpha=0.7)
plt.bar(x + 2 * bar_width, recall_scores, width=bar_width, label="Recall", align="center", alpha=0.7)
plt.bar(x + 3 * bar_width, f1_scores, width=bar_width, label="F1-Score", align="center", alpha=0.7)

plt.xlabel("Modelos")
plt.xticks(x + bar_width, models)
plt.title("Comparación de Métricas de Evaluación")
plt.legend(loc="lower right")
plt.show()


###Asterisco: Curva ROC-AUC

La curva ROC (Receiver Operating Characteristic) y el área bajo la curva (AUC) se utilizan para evaluar el rendimiento del modelo. Es una manera de ver la capacidad del modelo de identificar correctamente la característica de interés.

In [ ]:
#Ejemplo de funciones a utilizar para la propuesta de curva ROC-AUC.
from sklearn.metrics import roc_curve, roc_auc_score

y_scores = model.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
auc = roc_auc_score(y_test, y_scores)

plt.plot(fpr, tpr, label=f'AUC = {auc}')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()


In [ ]:
#Ejemplo de ROC-AUC
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Generamos datos de ejemplo
X, y = make_classification(n_samples=1000, n_features=20, random_state=672)

# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=568)

# Modelo con alto rendimiento (alto ROC-AUC)
model_high_auc = RandomForestClassifier(n_estimators=100, random_state=2664)
model_high_auc.fit(X_train, y_train)
y_pred_high_auc = model_high_auc.predict_proba(X_test)[:, 1]  # Probabilidad de clase positiva

# Modelo con bajo rendimiento (bajo ROC-AUC)
model_low_auc = LogisticRegression()
model_low_auc.fit(X_train, y_train)
y_pred_low_auc = model_low_auc.predict_proba(X_test)[:, 1]  # Probabilidad de clase positiva

# Calculamos las curvas ROC y el ROC-AUC para ambos modelos
fpr_high_auc, tpr_high_auc, _ = roc_curve(y_test, y_pred_high_auc)
roc_auc_high_auc = roc_auc_score(y_test, y_pred_high_auc)

fpr_low_auc, tpr_low_auc, _ = roc_curve(y_test, y_pred_low_auc)
roc_auc_low_auc = roc_auc_score(y_test, y_pred_low_auc)

# Visualizamos las curvas ROC
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(fpr_high_auc, tpr_high_auc, color='darkorange', lw=2, label='Modelo de Alto ROC-AUC (AUC = %0.2f)' % roc_auc_high_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - Random Forest')
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(fpr_low_auc, tpr_low_auc, color='darkorange', lw=2, label='Modelo de Bajo ROC-AUC (AUC = %0.2f)' % roc_auc_low_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - Regresión Logística')
plt.legend(loc='lower right')

plt.show()


## Evaluación en Regresión
Aunque hasta ahora hemos estado hablando de ciertas técnicas y algoritmos de clasificación, también tenemos que hablar de modelos de regresión para la predicción del comportamiento de una variable respuesta específica. A través de medidas como el MSE (Error Cuadrado Medio), el MAE (Error Cuadrado Absoluto) y el $R^2$ (coeficiente de determinación).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Cargamos el conjunto de datos "Diabetes"
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamos tres modelos diferentes
model_linear = LinearRegression()
model_svr = SVR(kernel='linear')
model_tree = DecisionTreeRegressor(max_depth=5)

model_linear.fit(X_train, y_train)
model_svr.fit(X_train, y_train)
model_tree.fit(X_train, y_train)

# Realizamos predicciones en el conjunto de prueba
y_pred_linear = model_linear.predict(X_test)
y_pred_svr = model_svr.predict(X_test)
y_pred_tree = model_tree.predict(X_test)

# Calculamos métricas de evaluación
mse_linear = mean_squared_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

mse_svr = mean_squared_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)

mse_tree = mean_squared_error(y_test, y_pred_tree)
r2_tree = r2_score(y_test, y_pred_tree)

# Visualizamos los resultados
plt.figure(figsize=(12, 5))
plt.subplot(1, 3, 1)
plt.scatter(y_test, y_pred_linear)
#plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--k')  # Línea diagonal
plt.xlabel("Valores Reales")
plt.ylabel("Predicciones")
plt.title(f"Regresión Lineal\nMSE: {mse_linear:.2f}, R^2: {r2_linear:.2f}")

plt.subplot(1, 3, 2)
plt.scatter(y_test, y_pred_svr)
#plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--k')  # Línea diagonal
plt.xlabel("Valores Reales")
plt.ylabel("Predicciones")
plt.title(f"Support Vector Machine (SVR)\nMSE: {mse_svr:.2f}, R^2: {r2_svr:.2f}")

plt.subplot(1, 3, 3)
plt.scatter(y_test, y_pred_tree)
#plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--k')  # Línea diagonal
plt.xlabel("Valores Reales")
plt.ylabel("Predicciones")
plt.title(f"Árbol de Decisión\nMSE: {mse_tree:.2f}, R^2: {r2_tree:.2f}")

plt.tight_layout()
plt.show()


##Consideraciones de Recursos Computacionales

Espero que hayan leído el capítulo que se les dejó en el control de lectura.

Las consideraciones de recursos son un aspecto crítico en la selección de algoritmos.


Consideraciones que deben tener:

**Recursos Computacionales:** La cantidad de recursos computacionales, como la potencia de cómputo y la memoria disponible, puede influir en la elección del algoritmo.

**Tiempo de Entrenamiento:** Si tienes limitaciones de tiempo para obtener resultados, es importante seleccionar algoritmos que se ajusten a tu cronograma.

**Escalabilidad:** La capacidad de un algoritmo para manejar conjuntos de datos de diferentes tamaños.

**Requisitos de Memoria:** Algunos algoritmos pueden requerir grandes cantidades de memoria para cargar y procesar conjuntos de datos.

**Condiciones de Producción:** Debes considerar si los resultados tienen una cierta necesidad de presentación con periodicidad específica.

**Algoritmos Paralelizables:** Si tienes acceso a una infraestructura de cómputo paralelo o distribuido, puedes considerar algoritmos que se pueden paralelizar fácilmente para acelerar el entrenamiento.

**Optimización de Hiperparámetros:** Algunos algoritmos pueden requerir una optimización extensa de hiperparámetros (Bayes, por ejemplo) para lograr un buen rendimiento.

**Consideraciones de Almacenamiento:** Modelos complejos pueden ocupar más espacio de almacenamiento, y debes asegurarte de tener suficiente capacidad de almacenamiento disponible.

In [ ]:
import time
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import sys

# Cargamos el conjunto de datos "Breast Cancer"
data = load_breast_cancer()
X = data.data
y = data.target

# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Consideración 1: Tiempo de Entrenamiento
# Comparamos el tiempo de entrenamiento de dos modelos: Bosques Aleatorios y SVM.
start_time = time.time()

model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)

rf_training_time = time.time() - start_time

start_time = time.time()

model_svm = SVC(kernel='linear')
model_svm.fit(X_train, y_train)

svm_training_time = time.time() - start_time

print(f"Tiempo de entrenamiento (Bosques Aleatorios): {rf_training_time:.2f} segundos")
print(f"Tiempo de entrenamiento (SVM): {svm_training_time:.2f} segundos")

# Consideración 2: Requisitos de Memoria
# Comparamos los requisitos de memoria para los dos modelos.
rf_memory_usage = sys.getsizeof(model_rf)
svm_memory_usage = sys.getsizeof(model_svm)

print(f"Requisitos de memoria (Bosques Aleatorios): {rf_memory_usage / (1024 * 1024):.2f} MB")
print(f"Requisitos de memoria (SVM): {svm_memory_usage / (1024 * 1024):.2f} MB")
print(f"Requisitos de memoria (Random Forests en bytes): {rf_memory_usage} bytes")
print(f"Requisitos de memoria (SVM en bytes): {svm_memory_usage} bytes")